In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("trying").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 11:17:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [5]:
import pp_events as pp

In [6]:
events_shot = pp.preprocessing(events)

In [7]:
events_shot.show()

+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+
|                  id|player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|goal|
+--------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+
|c7168289-8ba6-427...|  30111.0|          111.6|           34.7|          9.9323| 39.113426|                  0|             1|              1|              0|              0|             0|                   0|       0.05332383|       Off T|   0|
|aa63a13

In [8]:
#features = ['distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure']
features = ['shot_location_x', 'shot_location_y','distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble']
target = ['goal']

In [9]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)

In [10]:
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [11]:
from pyspark.ml.classification import LogisticRegression

# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)

In [12]:
from pyspark.ml import Pipeline

# Create the pipeline
pipeline = Pipeline(stages=[log_reg])

In [13]:
xg_model = pipeline.fit(train_data)

24/12/23 11:19:48 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [14]:
# Make predictions
predictions = xg_model.transform(test_data)

# Evaluate using BinaryClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.7839940271104228


In [15]:
predictions.show(truncate=False)

+------------------------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+--------------------------------------------------------------------------------------------+----------------------------------------+-----------------------------------------+----------+
|id                                  |player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|goal|features_vector                                                                             |rawPrediction                           |probability                              |prediction|
+------------------------------------+---------+---------------+---------------+----------------+-

In [16]:
from pyspark.sql.functions import col, udf, format_number, abs
from pyspark.sql.types import DoubleType

# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference_abs", abs(col('difference')))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference",'difference_abs').show(50,False)

+-----------------+----------------+-------------+--------------+
|shot_statsbomb_xg|goal_probability|difference   |difference_abs|
+-----------------+----------------+-------------+--------------+
|0.10166888       |0.1881344852    |-0.0864656052|0.0864656052  |
|0.064399794      |0.1806771442    |-0.1162773502|0.1162773502  |
|0.033228997      |0.0756254290    |-0.0423964320|0.042396432   |
|0.0447311        |0.1519337944    |-0.1072026944|0.1072026944  |
|0.10890295       |0.1645404323    |-0.0556374823|0.0556374823  |
|0.03492738       |0.0883850116    |-0.0534576316|0.0534576316  |
|0.0638635        |0.0932847323    |-0.0294212323|0.0294212323  |
|0.24384445       |0.0795066491    |0.1643378009 |0.1643378009  |
|0.033889998      |0.0834683726    |-0.0495783746|0.0495783746  |
|0.30627033       |0.1706786092    |0.1355917208 |0.1355917208  |
|0.038757622      |0.0672000455    |-0.0284424235|0.0284424235  |
|0.030633146      |0.0189920522    |0.0116410938 |0.0116410938  |
|0.1491743

In [18]:
# average of the difference
from pyspark.sql.functions import avg

avg_diff = predictions_with_goal_prob.select(avg(col("difference"))).collect()[0][0]
print(f"Average difference: {avg_diff}")

abs_avg_diff = predictions_with_goal_prob.select(avg(col("difference_abs"))).collect()[0][0]
print(f"Abs average difference: {abs_avg_diff}")

Average difference: -0.005542601315472378


Abs average difference: 0.05229424852629077


In [22]:
# Average difference    : -0.005542601315472378
# Abs average difference:  0.05229424852629077

In [19]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model

from pyspark.sql.functions import round

predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

predictions_with_goal_prob.select("goal", "goal_probability_rounded", "shot_statsbomb_xg_rounded").show()

+----+------------------------+-------------------------+
|goal|goal_probability_rounded|shot_statsbomb_xg_rounded|
+----+------------------------+-------------------------+
|   0|                     0.0|                      0.0|
|   1|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   1|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   0|                     0.0|                      0.0|
|   1|        

In [20]:
predictions_with_goal_prob.groupBy("goal").count().show()

+----+-----+
|goal|count|
+----+-----+
|   1| 1326|
|   0|10857|
+----+-----+



In [21]:
from pyspark.sql.functions import col, when

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0)
).withColumn(
    "statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0)
)

In [22]:
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")

Model Accuracy: 89.40%
StatsBomb xG Accuracy: 90.42%


In [23]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

shot_location_x: -0.07089985223630016
shot_location_y: -0.004657175090242295
distance_to_goal: -0.2131395419906175
shot_angle: 0.0119234592952614
preferred_foot_shot: 0.9855072780022439
under_pressure: -0.08668524824306786
shot_aerial_won: -1.1647390169920981
shot_first_time: -0.3580628236898425
shot_one_on_one: 0.41576264305295624
shot_open_goal: 1.4443852791674474
shot_follows_dribble: 1.5189010520536304
